# Merging WU Weather Data (30 mins weather interval)
## by : Isaac Harris

In [16]:
import pandas as pd
import glob
import os.path
from pathlib import Path

In [17]:
os.chdir("../Data/raw/WU")
# Create a list of all CSV files
files = glob.glob("*.csv")

# Create an empty list to append the df
filenames = []

for csv in files:
    df = pd.read_csv(csv)
    df['Date'] = os.path.basename(csv)
    filenames.append(df)

In [18]:
df= pd.concat(filenames)

In [19]:
df['Date'] = df['Date'].str.replace(r'\D', '')

/var/folders/x3/zqbjwpy10g30gjb8chvrw95h0000gn/T/ipykernel_16565/1463407949.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Date'] = df['Date'].str.replace(r'\D', '')


In [20]:
df['Datetime'] = df['Date'] + ' ' + df['Time']

In [21]:
df = df.drop(columns= ['Date', 'Time'])

In [22]:
df['Datetime'] = pd.to_datetime(df['Datetime'])

In [23]:
df.head()

,Temperature,Dew Point,Humidity,Wind,Wind Speed,Wind Gust,Pressure,Precip.,Condition,Datetime
0,59 °F,46 °F,63 %,W,15 mph,0 mph,29.46 in,0.0 in,Partly Cloudy,2009-05-22 00:00:00
1,59 °F,46 °F,63 %,W,18 mph,0 mph,29.46 in,0.0 in,Mostly Cloudy,2009-05-22 00:30:00
2,59 °F,43 °F,55 %,WNW,18 mph,33 mph,29.46 in,0.0 in,Mostly Cloudy,2009-05-22 00:54:00
3,59 °F,45 °F,59 %,W,21 mph,33 mph,29.46 in,0.0 in,Mostly Cloudy / Windy,2009-05-22 01:00:00
4,57 °F,45 °F,63 %,W,18 mph,31 mph,29.49 in,0.0 in,Mostly Cloudy,2009-05-22 01:30:00


In [24]:
sel_col = ['Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 'Wind Gust', 'Pressure', 'Precip.']

for col in sel_col: 
    df[col] = df[col].str.extract('(\d+)', expand=False)

In [25]:
df.isna().sum()

Temperature      44
Dew Point      1058
Humidity         70
Wind             58
Wind Speed     2793
Wind Gust         0
Pressure         24
Precip.           0
Condition         6
Datetime          0
dtype: int64

In [26]:
col_has_na = ['Temperature', 'Dew Point', 'Humidity', 'Wind', 'Wind Speed', 'Pressure', 'Condition']

for col in col_has_na:
    df[col] = df[col].fillna(method = 'bfill')

df.isna().sum()    

Temperature    0
Dew Point      0
Humidity       0
Wind           0
Wind Speed     0
Wind Gust      0
Pressure       0
Precip.        0
Condition      0
Datetime       0
dtype: int64

In [27]:
df['Temperature'] = round((df['Temperature'].astype(int) - 32) / (9/5),2)
df['Dew Point'] = round((df['Dew Point'].astype(int) - 32) / (9/5),2)

In [28]:
df_final = df.rename(columns = {'Temperature' :'temperature(celcius)', 'Dew Point': 'dew_point(celcius)', 'Humidity' :'humidity(%)',
                                        'Wind Speed': 'wind_speed(mph)', 'Wind Gust': 'wind_gush(mph)', 'Pressure': 'pressure(in)', 'Precip.' : 'precip.(in)'})

In [29]:
df_final.head()

,temperature(celcius),dew_point(celcius),humidity(%),Wind,wind_speed(mph),wind_gush(mph),pressure(in),precip.(in),Condition,Datetime
0,15.00,7.78,63,W,15,0,29,0,Partly Cloudy,2009-05-22 00:00:00
1,15.00,7.78,63,W,18,0,29,0,Mostly Cloudy,2009-05-22 00:30:00
2,15.00,6.11,55,WNW,18,33,29,0,Mostly Cloudy,2009-05-22 00:54:00
3,15.00,7.22,59,W,21,33,29,0,Mostly Cloudy / Windy,2009-05-22 01:00:00
4,13.89,7.22,63,W,18,31,29,0,Mostly Cloudy,2009-05-22 01:30:00


In [30]:
path = Path(os.getcwd())
levels_up = 1
to_folder = path.parents[levels_up]
os.chdir(to_folder)
df.to_csv( "edited/WU_merged/wu_weather_merged.csv", index=False, encoding='utf-8-sig')